In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

4대궁 데이터 전처리

In [38]:
palace2023 = pd.read_csv("./4대궁2023.csv", encoding='cp949')

# 궁 리스트
palaces = ['경복궁', '덕수궁', '창경궁', '창덕궁']
# 각 궁별로 2023년 방문객 정리
for palace in palaces:
  df = palace2023[f'{palace}(유료)'] + palace2023[f'{palace}(무료)']
  df.to_csv(f'{palace}_2023_방문객.csv', index=False)

os.remove('./4대궁2023.csv')

서울대공원 데이터 전처리<br>
파일별로 시트명이 다르게(특정 달에 공백 존재) 저장되어 있어 다른 셀에서 작업함

In [9]:
seoul_park_xls = pd.ExcelFile('./서울대공원2023.xlsx')
month = ['1월', '2월 ', '3월 ', '4월 ', '5월 ', '6월 ', '7월', '8월', '9월', '10월', '11월', '12월']

# 각 시트에서 특정 열만 가져와서 리스트에 저장
dfs = []
for sheet in month:
  seoul_park = pd.read_excel(seoul_park_xls, sheet_name=sheet, header=4)
  seoul_park.drop(0, inplace=True)
  df = seoul_park[['날짜', '일합계']]  # 원하는 열만 가져오기
  dfs.append(df)

# 모든 시트의 특정 열을 세로로 연결
merged_df = pd.concat(dfs, ignore_index=True)
# 새로운 엑셀 파일로 저장 (기존 파일 수정하려면 다른 방식 필요)
merged_df.to_csv("merged_seoul_park_2023.csv", index=False)

os.remove('./서울대공원2023.xlsx')

In [10]:
seoul_park_xls = pd.ExcelFile('./서울대공원2024.xlsx')
month = ['1월', '2월 ', '3월 ', '4월 ', '5월', '6월', '7월', '8월', '9월', '10월', '11월', '12월']

# 각 시트에서 특정 열만 가져와서 리스트에 저장
dfs = []
for sheet in month:
  seoul_park = pd.read_excel(seoul_park_xls, sheet_name=sheet, header=4)
  seoul_park.drop(0, inplace=True)
  df = seoul_park[['날짜', '일합계']]  # 원하는 열만 가져오기
  dfs.append(df)

# 모든 시트의 특정 열을 세로로 연결
merged_df = pd.concat(dfs, ignore_index=True)
# 새로운 엑셀 파일로 저장 (기존 파일 수정하려면 다른 방식 필요)
merged_df.to_csv("merged_seoul_park_2024.csv", index=False)

os.remove('./서울대공원2024.xlsx')

In [11]:
dfs = []
df1 = pd.read_csv('./merged_seoul_park_2023.csv')
df2 = pd.read_csv('./merged_seoul_park_2024.csv')
dfs.append(df1)
dfs.append(df2)

merged_seoul_park = pd.concat(dfs, ignore_index=True)
merged_seoul_park.to_csv('merged_seoul_park.csv', index=False)

os.remove('./merged_seoul_park_2023.csv')
os.remove('./merged_seoul_park_2024.csv')

feature_dataset 따로 제작

In [35]:
temp = pd.read_excel('./proceed_temperature_seoul.xlsx')['Temperature']
wind = pd.read_excel('./proceed_wind_seoul.xlsx', header=1)['평균풍속(m/s)']
humidity = pd.read_excel('./processed_humidity_data.xlsx', header=1)['평균습도(%rh)']
rainfall = pd.read_excel('./processed_rainfall_data.xlsx', header=1)['강수량(mm)']
# exchamnge_rate = pd.read_csv('./processed_exchange_rate_data.csv') 어떻게 해야 되지..


df_features = pd.concat([temp, wind, humidity, rainfall], axis=1)
df_features

,Temperature,평균풍속(m/s),평균습도(%rh),강수량(mm)
0,-0.2,2.7,54.5,0.0
1,-4.5,2.5,45.9,0.0
2,-5.0,1.8,49.0,0.0
3,-1.8,1.9,51.4,0.0
4,-1.6,1.6,58.1,0.0
...,...,...,...,...
726,-2.6,2.7,51.4,0.0
727,-3.0,2.5,48.3,0.0
728,1.1,1.5,56.1,0.0
729,5.5,2.3,68.5,0.0


#경복궁

In [ ]:
df = pd.read_csv('./경복궁_2023_방문객.csv')
merged_df = pd.concat([df, df_features], axis=1)

X = df_features[['Temperature', '평균풍속(m/s)', '평균습도(%rh)', '강수량(mm)']]  # 입력 특성
y = df
# 훈련, 테스트 데이터셋 분할
train_size = int(len(df) * 0.8)  # 80%를 훈련 데이터로 사용
train = merged_df.loc[:train_size]
test = merged_df.loc[train_size:]

X_train = train[['Temperature', '평균풍속(m/s)', '평균습도(%rh)', '강수량(mm)']]
y_train = train.iloc[:, 0]
X_test = test[['Temperature', '평균풍속(m/s)', '평균습도(%rh)', '강수량(mm)']]
y_test = test.iloc[:, 0]


In [48]:
# AdaBoost 모델 학습
adaboost = AdaBoostRegressor(n_estimators=50)
adaboost.fit(X_train, y_train)

# 예측 및 평가
y_pred = adaboost.predict(X_test)

In [ ]:
# XGBoost 모델 학습
xgb = XGBRegressor

#서울대공원

In [52]:
df = pd.read_csv('./merged_seoul_park.csv')['일합계']
merged_df = pd.concat([df, df_features], axis=1)

X = df_features[['Temperature', '평균풍속(m/s)', '평균습도(%rh)', '강수량(mm)']]  # 입력 특성
y = df
# 훈련, 테스트 데이터셋 분할
train_size = int(len(df) * 0.8)  # 80%를 훈련 데이터로 사용
train = merged_df.loc[:train_size]
test = merged_df.loc[train_size:]

X_train = train[['Temperature', '평균풍속(m/s)', '평균습도(%rh)', '강수량(mm)']]
y_train = train.iloc[:, 0]
X_test = test[['Temperature', '평균풍속(m/s)', '평균습도(%rh)', '강수량(mm)']]
y_test = test.iloc[:, 0]

df

,일합계
0,1268.0
1,365.0
2,345.0
3,428.0
4,439.0
...,...
729,709.0
730,715.0
731,1156.0
732,19266.0


In [53]:
# AdaBoost 모델 학습
adaboost = AdaBoostRegressor(n_estimators=50)
adaboost.fit(X_train, y_train)

# 예측 및 평가
y_pred = adaboost.predict(X_test)

ValueError: Input X contains NaN.
AdaBoostRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values